# Distribution Matching Model

In this notebook we attempt to fit the distribution of crate data to match the distribution of the thin-section data.

In [1]:
from spectral.io import envi

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import joblib

import keras
import tensorflow as tf
print(f"keras version: {keras.__version__}")
print(f"tensorflow version: {tf.__version__}")

from source.neuralnet import AutoEncoder
from source.distributionnet import DistributionalEncoder
from source.hyperspectrum_models import Masker

Using TensorFlow backend.


keras version: 2.3.1
tensorflow version: 2.1.0


In [2]:
np.random.seed(1234)
from tensorflow_core.random import set_seed
set_seed(1234)

In [3]:
img_c195 = envi.open("data/VNIR and SWIR interval/CMM-111_195_extracted_image.hdr", "data/VNIR and SWIR interval/CMM-111_195_extracted_image.img")
img_c196 = envi.open("data/VNIR and SWIR interval/CMM-111_196_extracted_image.hdr", "data/VNIR and SWIR interval/CMM-111_196_extracted_image.img")

In [4]:
wl_c = np.array( img_c195.bands.centers )
n_bands_c = img_c195.shape[-1]

hypr_c195 = img_c195.load()
print(f"crate image 195 shape: {hypr_c195.shape}")

masker = joblib.load('binaries/masker.joblib')
mask_c195 = masker(hypr_c195)
hypr_c195 = hypr_c195[mask_c195,:]

hypr_c195 = hypr_c195.reshape(-1, n_bands_c)

hypr_c196 = img_c196.load()
print(f"crate image 196 shape: {hypr_c196.shape}")

mask_c196 = masker(hypr_c196)
hypr_c196 = hypr_c196[mask_c196,:]

hypr_c196 = hypr_c196.reshape(-1, n_bands_c)

crate image 195 shape: (527, 337, 411)
crate image 196 shape: (525, 323, 411)


In [5]:
hypr_t = pd.read_pickle('data/original_data/hyper[16_17_20].pkl').values

In [6]:
sc = StandardScaler()
Ht = sc.fit_transform(hypr_t)

ae = AutoEncoder(n_components=5)
ae.fit(Ht, epochs=1, batch_size=200)

Ht = ae.transform(Ht)

Epoch 1/1
15319/15319 [==============================] - 1s 50us/step - loss: 0.2359


In [7]:
sc_c = StandardScaler()
Hc = sc_c.fit_transform(hypr_c195)

dae = DistributionalEncoder()
dae.fit(Hc, Ht)

AttributeError: 'samplingLayer' object has no attribute 'name'